In [1]:
import glob
import os
import pandas as pd
import geopy
from geopy.extra.rate_limiter import RateLimiter
import gitdir
from geopy.geocoders import Nominatim
locator = Nominatim(user_agent="myGeocoder")
import IPython
import svn

In [7]:
!gitdir https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

Downloaded: .gitignore
Downloaded: 01-22-2020.csv
Downloaded: 01-23-2020.csv
Downloaded: 01-24-2020.csv
Downloaded: 01-25-2020.csv
Downloaded: 01-26-2020.csv
Downloaded: 01-27-2020.csv
Downloaded: 01-28-2020.csv
Downloaded: 01-29-2020.csv
Downloaded: 01-30-2020.csv
Downloaded: 01-31-2020.csv
Downloaded: 02-01-2020.csv
Downloaded: 02-02-2020.csv
Downloaded: 02-03-2020.csv
Downloaded: 02-04-2020.csv
Downloaded: 02-05-2020.csv
Downloaded: 02-06-2020.csv
Downloaded: 02-07-2020.csv
Downloaded: 02-08-2020.csv
Downloaded: 02-09-2020.csv
Downloaded: 02-10-2020.csv
Downloaded: 02-11-2020.csv
Downloaded: 02-12-2020.csv
Downloaded: 02-13-2020.csv
Downloaded: 02-14-2020.csv
Downloaded: 02-15-2020.csv
Downloaded: 02-16-2020.csv
Downloaded: 02-17-2020.csv
Downloaded: 02-18-2020.csv
Downloaded: 02-19-2020.csv
Downloaded: 02-20-2020.csv
Downloaded: 02-21-2020.csv
Downloaded: 02-22-2020.csv
Downloaded: 02-23-2020.csv
Downloaded: 02-24-2020.csv
Downloaded: 02-25-2020.csv
Downloaded: 02-26-2020.csv
Downl

Traceback (most recent call last):
  File "c:\users\pxppa\appdata\local\programs\python\python36\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\users\pxppa\appdata\local\programs\python\python36\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\pxppa\AppData\Local\Programs\Python\Python36\Scripts\gitdir.exe\__main__.py", line 5, in <module>
  File "c:\users\pxppa\appdata\local\programs\python\python36\lib\site-packages\gitdir\__main__.py", line 3, in <module>
    main()
  File "c:\users\pxppa\appdata\local\programs\python\python36\lib\site-packages\gitdir\gitdir.py", line 152, in main
    print_text("\u2714 Download complete", "green", in_place=True)
  File "c:\users\pxppa\appdata\local\programs\python\python36\lib\site-packages\gitdir\gitdir.py", line 30, in print_text
    print(COLOR_NAME_TO_CODE[color] + text + Style.RESET_ALL, **kwargs)
  File "c:\users\pxppa\appdata\local\programs\python\python36\lib\site-packag

In [8]:
# downloade latest files
#!gitdir https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

# Parse out MDY from filenames, iterate and append all files.
filenames = []
os.chdir("C:/Users/pxppa/OneDrive/Documents/PowerBI Projects/Corona Virus/csse_covid_19_data/csse_covid_19_daily_reports")

for file in glob.glob('*.csv'):
    month, day, year = file.split('.')[0].split('-') 
    combined = (pd.read_csv(file).assign(Month=month, Day=day, Year=year))
    filenames.append(combined)
    
combined = pd.concat(filenames)

#Show me the datatypes
datatypes = combined.dtypes
print(datatypes)

#Delete useless columns -- aka, not persisting columns
del combined['Latitude']
del combined['Longitude']

# Change Mainland China to China -- geocoding bugs out
combined.loc[(combined['Country/Region'] == 'Mainland China'),'Country/Region' ] = 'China'

# Keep only distinct countries/regions
Temp = combined.drop_duplicates(["Country/Region"].copy())

# 1 - conveneint function to delay between geocoding calls
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)

# 2- - create location column
Temp['location'] = Temp['Country/Region'].apply(geocode)

# 3 - create longitude, laatitude and altitude from location column (returns tuple)
Temp['point'] = Temp['location'].apply(lambda loc: tuple(loc.point) if loc else None)

# 4 - split point column into latitude, longitude and altitude columns
Temp[['latitude', 'longitude', 'altitude']] = pd.DataFrame(Temp['point'].tolist(), index=Temp.index)

# Select only subset of columns
Temp1 = Temp[['Country/Region','latitude','longitude']]

# Left join to original dataframe
combined = combined.merge(Temp1, left_on='Country/Region', right_on='Country/Region', how='left')

# Date field
combined['Date'] = pd.to_datetime(combined[['Year', 'Month', 'Day']])

# Create new dataframe - for joining purposes
joiner = combined.copy()

# Create an offset date variable so that I may calculate the daily numbers, rather than cumulative
joiner['Date offset'] = joiner['Date'] + pd.Timedelta(days=1)

# Rename my helper columns
joiner.rename(columns={'Confirmed': 'Confirmed offset'
                       , 'Deaths': 'Deaths offset'
                       , 'Recovered': 'Recovered offset'}
                       , inplace=True)

# Only use the subset of columns
joiner = joiner[['Province/State'
                 ,'Country/Region'
                 ,'Confirmed offset'
                 ,'Recovered offset'
                 ,'Deaths offset'
                 ,'Date offset']].copy()

# Left join to my master data using province, region and date -- but for date, match to the offset
Final_combined = pd.merge(combined
                           ,joiner
                           ,left_on=['Province/State', 'Country/Region','Date']
                           ,right_on=['Province/State', 'Country/Region','Date offset']
                          ,how='left'
                           )

# Calculate the daily numbers
Final_combined['Confirmed Daily'] = Final_combined['Confirmed'] - Final_combined['Confirmed offset']
Final_combined['Dead Daily'] = Final_combined['Deaths'] - Final_combined['Deaths offset']
Final_combined['Recovered Daily'] = Final_combined['Recovered'] - Final_combined['Recovered offset']

# Delete -now- useless columns
del Final_combined['Confirmed offset']
del Final_combined['Deaths offset']
del Final_combined['Recovered offset']
del Final_combined['Date offset']

# Export dataset
os.chdir("C:/Users/pxppa/OneDrive/Documents/PowerBI Projects/Corona Virus")
Final_combined.to_csv("Final Dataset/combined.csv")

combined.head(2)

Confirmed         float64
Country/Region     object
Day                object
Deaths            float64
Last Update        object
Latitude          float64
Longitude         float64
Month              object
Province/State     object
Recovered         float64
Year               object
dtype: object


c:\users\pxppa\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
c:\users\pxppa\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
c:\users\pxppa\appdata\local\programs\python\python36\lib\site-packages\pandas\core\frame.py:2540: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveat

,Confirmed,Country/Region,Day,Deaths,Last Update,Month,Province/State,Recovered,Year,latitude,longitude,Date
0,1.0,China,22,NaN,1/22/2020 17:00,01,Anhui,NaN,2020,35.000074,104.999927,2020-01-22
1,14.0,China,22,NaN,1/22/2020 17:00,01,Beijing,NaN,2020,35.000074,104.999927,2020-01-22
